In [172]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import joblib
import emoji
import re
import nltk
import spacy
from spacy import displacy
from annoy import AnnoyIndex
from collections import Counter

from tqdm.notebook import tqdm
tqdm.pandas()

In [21]:
pathModel="model/"

def load_vectorizer(filename):
    '''Функция для загрузки модели векторизатора'''
    tfidf = TfidfVectorizer()
    tfidf = joblib.load(pathModel+filename)
    return tfidf

def load_logreg(filename):
    '''Функция для загрузки модели классификатора'''
    logit = LogisticRegression()
    logit = joblib.load(pathModel+filename)
    return logit

def load_stopwords():
    ''' подготовка моделей обработки текста '''
    nltk.download('stopwords')
    stopwords_nltk = nltk.corpus.stopwords.words('russian') #лист русских стоп-слов
    stopwords_nltk_en = nltk.corpus.stopwords.words('english')
    stopwords_nltk.extend(stopwords_nltk_en) #часть текста на английском
    new_stop = ['здравствовать', 'подсказать', 'сказать', "пожалуйста", "спасибо",  "благодарить", "извинить",
                'вопрос', 'тема', "ответ", "ответить", "почему", "что", "уважаемый", "нынче", "пжл", "анон",
                'которая', 'которой', 'которую', 'которые', 'который', 'которых', 'это', "мочь", "её",
                "нам", "наш", "всем", "здраствуйте", "типо",
                'вообще', "всё", "весь", "ещё", "просто",  "якобы", "причём", 'точно', "хотя", "именно", 'неужели',
                "т","е", "п", "д", "г", "ул", "город", "улица", "br", "none", "https", "#"]  #специфичные стоп-слов
    stopwords_nltk.extend(new_stop)
    return stopwords_nltk

def load_nlp():
    nlp = spacy.load('ru_core_news_md')
    return nlp

def load_upload(uploaded_file):
    try:
        data = pd.read_csv(uploaded_file, sep=',')
    except:
        data = pd.read_csv(uploaded_file, sep=';')
    return data

In [29]:
def full_clean(text):
    '''очистка строки текста'''
    text = emoji.demojize(text)
    text=re.sub(r"[^a-zA-Zа-яА-ЯёЁ0-9#]", " ", text)
    text = text.lower()
    text = re.sub(" +", " ", text) #оставляем только 1 пробел
    text = text.replace("добрый день", "").replace("добрый вечер", "").replace("доброе утро", "").replace("сообщение без текста", "").replace("да подтверждаю", "").replace("до сих пор", "")
    text = text.strip()
    #токены для моделей
    tokens = [token.lemma_ for token in nlp(text) if token.lemma_ not in stopwords_nltk]
    #для tfidf на вход текст
    text = " ".join(tokens)
    return text, tokens

def preprocess_text(df):
    '''подготовка текста к подаче в модель колонкой'''
    new_corpus = []
    new_tokens = []

    for text in tqdm(df):
        text, tokens = full_clean(text)
        new_corpus.append(text)
        new_tokens.append(tokens)
    return new_corpus, new_tokens

# Функция для классификации данных
def classify_data(logit, data):
    ''' предсказание логистической регрессией для вектора предложения '''
    return logit.predict(data)

In [30]:
def tfidf_fit(train=None, test=None, tfidf=True, ngram_range=(1, 1), max_features=1000, save=False):
    #на вход текст
    #min_df : игнорируются термины, частота употребления которых строго ниже заданного порога.
    #max_df : игнорируются термины, частота которых строго превышает заданный порог
    if test:
        data = pd.concat([train, test])
    else:
        data = train
    if tfidf:        
        model = TfidfVectorizer(ngram_range=(1, 1), max_features=max_features, analyzer='word', #max_df = 0.9,
                            lowercase = False, sublinear_tf=True)
    else:
        model = CountVectorizer(max_features=max_features) 
    #тренировка
    model.fit(data)
    #feature_names = model.get_feature_names_out()
    
    #сохранение натренированной модели для приложения
    if save:
        joblib.dump(model, 'tfidf.pkl') 
        
    return model

def tfidf_embeding(model=None, df=None):
    '''Преобразование текста в мешок слов'''
    X = model.transform(df)
    return X.toarray()

In [170]:
def index_annoy(X_train_part):
    #построение индекса
    #"angular", "euclidean", "manhattan", "hamming", or "dot" 384 768 1024
    #emb_len = X_train_part.shape[1]
    emb_len=len(X_train_part[0])
    t = AnnoyIndex(emb_len, metric = 'angular')

    # train['id'], 
    for user_id, user_embedding in enumerate(X_train_part):
        t.add_item(user_id, user_embedding)
    t.build(-1)
    
    return t

def predict_nns_(t, X_valid, count=1):
    #через apply
    idx = t.get_nns_by_vector(X_valid, count, search_k=-1)  
    return train_part[idx].values[0]

def predict_nns(t, X_valid, y_train_part, count=1):
    #для массива срезу
    predict=[]
    for emb in tqdm(X_valid):
        if count==1:
            idx = t.get_nns_by_vector(emb, count, search_k=-1) 
            predict.append(y_train_part[idx[0]])
        elif count==3:
            x=[y_train_part[idx[0]], y_train_part[idx[1]], y_train_part[idx[2]]]
            predict.append(Counter(x).most_common()[0][0])
    return predict

In [151]:
def postpredict(model, t, X_valid, y_train_part, lim=0.1):
    #если уверенность в метке низкая ииспользуем другой классификатор
    
    #t = index_annoy(X_train_part)
    logit_class = list(model.classes_)
    
    model_valid = model.predict(X_valid)
    model_valid_ = model.predict_proba(X_valid)
    
    scores = []
    model_valid_new=[]
    lim=lim
    for i in range(len(model_valid)):
        #класс, вероятность
        #print(model_valid[i], model_valid_[i][logit_class.index(model_valid[i])])
        score = model_valid_[i][logit_class.index(model_valid[i])]
        scores.append(score)
        if score >= lim:
            model_valid_new.append(model_valid[i])
        elif score < lim:
            idx = t.get_nns_by_vector(X_valid[i], 1, search_k=-1)[0] 
            model_valid_new.append(y_train_part[idx])
            #model_valid_new.append(catc.predict(X_valid[i])[0])
            
    return model_valid_new

In [22]:
tfidf = load_vectorizer('tfidf.pkl')
logit_group = load_logreg("logit_group.sav")
logit_title = load_logreg("logit_title.sav")
logit_otdel = load_logreg("logit_otdel.sav")
stopwords_nltk = load_stopwords()
nlp = load_nlp()

[nltk_data] Downloading package stopwords to /home/tanya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Загрузим и предобработаем тренировочные данные

In [25]:
train = pd.read_csv('data/train_dataset_train.csv', sep=';', index_col=None)
train['text_clean'], train['tokens'] = preprocess_text(train['Текст инцидента'])
train.head()

  0%|          | 0/23128 [00:00<?, ?it/s]

,Исполнитель,Группа тем,Текст инцидента,Тема,text_clean,tokens
0,Лысьвенский городской округ,Благоустройство,"'Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах,сегодня 20 08 22 мать идти ленин дом 96 94 фон...,"[сегодня, 20, 08, 22, мать, идти, ленин, дом, ..."
1,Министерство социального развития ПК,Социальное обслуживание и защита,"'Пермь г, +79194692145. В Перми с ноября 2021 ...",Оказание гос. соц. помощи,пермь 79194692145 пермь ноябрь 2021 год работа...,"[пермь, 79194692145, пермь, ноябрь, 2021, год,..."
2,Министерство социального развития ПК,Социальное обслуживание и защита,'Добрый день ! Скажите пожалуйста если подовал...,Дети и многодетные семьи,подовала пособие 3 7 2 декабрь повторно подать...,"[подовала, пособие, 3, 7, 2, декабрь, повторно..."
3,Город Пермь,Общественный транспорт,'Каждая из них не о чем. Люди на остановках хо...,Содержание остановок,каждый человек остановка хотеть укрыться непог...,"[каждый, человек, остановка, хотеть, укрыться,..."
4,Министерство здравоохранения,Здравоохранение/Медицина,'В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу,березниках сын привитого откоронавируса заболе...,"[березниках, сын, привитого, откоронавируса, з..."


In [26]:
#сохраним предобработанный текст
train.to_csv("data/train_clean.csv", index=False)

In [27]:
#удалим пустые
train = train[train['text_clean']!=""]
train.shape

(22737, 6)

Построим векторы по тренировочным данным

In [178]:
tfidf = tfidf_fit(train=train['text_clean'], tfidf=True, max_features=10000)
feature_names = tfidf.get_feature_names_out()
len(feature_names)

10000

In [32]:
#joblib.dump(tfidf, 'model/tfidf.pkl')

['model/tfidf.pkl']

In [179]:
tfidf_embed = tfidf_embeding(model=tfidf, df=train['text_clean'])

In [180]:
y_train = train[['Исполнитель', 'Группа тем', 'Тема']].reset_index()

#### Загрузка тестовых

In [108]:
submission = pd.read_csv('data/submission.csv', sep=';', index_col=None)
submission

,id,Группа тем,Тема
0,0,Физическая культура и спорт,Строительство спортивной инфраструктуры
1,1,Физическая культура и спорт,Строительство спортивной инфраструктуры
2,2,Физическая культура и спорт,Строительство спортивной инфраструктуры
3,3,Физическая культура и спорт,Строительство спортивной инфраструктуры
4,4,Физическая культура и спорт,Строительство спортивной инфраструктуры
...,...,...,...
9738,9738,Физическая культура и спорт,Строительство спортивной инфраструктуры
9739,9739,Физическая культура и спорт,Строительство спортивной инфраструктуры
9740,9740,Физическая культура и спорт,Строительство спортивной инфраструктуры
9741,9741,Физическая культура и спорт,Строительство спортивной инфраструктуры


In [111]:
testname='data/test.csv'
testcol='Текст инцидента'

test = pd.read_csv(testname, sep=';', index_col=None)#[[testcol]]
test['text_clean'], test['tokens'] = preprocess_text(test[testcol])
test.head()

  0%|          | 0/9743 [00:00<?, ?it/s]

,id,Текст инцидента,text_clean,tokens
0,0,'Может создать петицию по нашей проблеме)..<br...,создать петиция проблема хоккейный коробка обя...,"[создать, петиция, проблема, хоккейный, коробк..."
1,1,"'Очень надеялась, что недоразумение с Декабрис...",очень надеяться недоразумение декабрист 2 испр...,"[очень, надеяться, недоразумение, декабрист, 2..."
2,2,'Хоть бы отремонтировали поликлинику на Гайве....,отремонтировать поликлиника гайве разу ремонт ...,"[отремонтировать, поликлиника, гайве, разу, ре..."
3,3,"'Добрый день! Получаю выплаты с 3 до 7 лет, по...",получать выплата 3 7 год подавать прошлый год ...,"[получать, выплата, 3, 7, год, подавать, прошл..."
4,4,'Добрый день! Подскажите пожалуйста куда обращ...,обращаться программа молодой семья,"[обращаться, программа, молодой, семья]"


In [181]:
test_embed = tfidf_embeding(model=tfidf, df=test['text_clean'])

Предсказание по фолдам

### Группа тем

In [182]:
#folds = KFold(n_splits=5, shuffle=True, random_state=42)
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
n_class=y_train['Группа тем'].nunique()#.values[0]
#количество сэмплов, количество классов
pred_proba = np.zeros((test_embed.shape[0], n_class))

y=np.array(y_train['Группа тем'])
logit = LogisticRegression(solver = "liblinear", C=2.0, class_weight="balanced", max_iter=200, multi_class="ovr")

In [183]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(tfidf_embed, y)):
    print("fold n°{}".format(fold_ + 1))
    X_train_part, y_train_part = tfidf_embed[trn_idx], y[trn_idx]
    X_valid, y_valid = tfidf_embed[val_idx], y[val_idx]
    
    logit.fit(X_train_part, y_train_part)    
    pred_proba += logit.predict_proba(test_embed) / folds.n_splits

fold n°1
fold n°2
fold n°3
fold n°4
fold n°5


In [184]:
predict_group = pd.DataFrame(pred_proba, columns=logit.classes_).idxmax(axis="columns").tolist()

### Темы

In [185]:
#folds = KFold(n_splits=5, shuffle=True, random_state=42)
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
n_class=y_train['Тема'].nunique()#.values[0]
#количество сэмплов, количество классов
pred_proba = np.zeros((test_embed.shape[0], n_class))

y=np.array(y_train['Тема'])
logit = LogisticRegression(solver = "liblinear", C=7.0, class_weight="balanced", max_iter=200, multi_class="ovr")

In [186]:
%%time
for fold_, (trn_idx, val_idx) in enumerate(folds.split(tfidf_embed, y)):
    print("fold n°{}".format(fold_ + 1))
    X_train_part, y_train_part = tfidf_embed[trn_idx], y[trn_idx]
    X_valid, y_valid = tfidf_embed[val_idx], y[val_idx]
    
    logit.fit(X_train_part, y_train_part)    
    pred_proba += logit.predict_proba(test_embed) / folds.n_splits

fold n°1
fold n°2
fold n°3
fold n°4
fold n°5
CPU times: user 4min 10s, sys: 8min 40s, total: 12min 51s
Wall time: 1min 7s


In [187]:
predict_title = pd.DataFrame(pred_proba, columns=logit.classes_).idxmax(axis="columns").tolist()

### Исполнители

In [81]:
#folds = KFold(n_splits=5, shuffle=True, random_state=42)
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
n_class=y_train['Исполнитель'].nunique()#.values[0]
#количество сэмплов, количество классов
pred_proba = np.zeros((test_embed.shape[0], n_class))

y=np.array(y_train['Исполнитель'])
logit = LogisticRegression(solver = "lbfgs", C=1.0, class_weight="balanced", max_iter=200, multi_class="ovr")

In [82]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(tfidf_embed, y)):
    print("fold n°{}".format(fold_ + 1))
    X_train_part, y_train_part = tfidf_embed[trn_idx], y[trn_idx]
    X_valid, y_valid = tfidf_embed[val_idx], y[val_idx]
    
    logit.fit(X_train_part, y_train_part)    
    pred_proba += logit.predict_proba(test_embed) / folds.n_splits

fold n°1
fold n°2
fold n°3
fold n°4
fold n°5


In [83]:
predict_otdel = pd.DataFrame(pred_proba, columns=logit.classes_).idxmax(axis="columns").tolist()

### Простые модели

Исполнители

In [85]:
logit_otdel = LogisticRegression(solver = "lbfgs", C=1.0, class_weight="balanced", max_iter=200, multi_class="ovr")
logit_otdel.fit(tfidf_embed, y_train['Исполнитель'])

LogisticRegression(class_weight='balanced', max_iter=200, multi_class='ovr')

In [86]:
# save the model to disk
#joblib.dump(logit_otdel, 'model/logit_otdel.sav')
 
# load the model from disk
#logit = joblib.load('model/logit_otdel.sav')

['model/logit_otdel.sav']

In [87]:
predict_otdel = logit_otdel.predict(test_embed)

Группы тем

In [88]:
logit_group = LogisticRegression(solver = "lbfgs", C=2.0, class_weight="balanced", max_iter=200, multi_class="ovr")
logit_group.fit(tfidf_embed, y_train["Группа тем"])

LogisticRegression(C=2.0, class_weight='balanced', max_iter=200,
                   multi_class='ovr')

In [90]:
# save the model to disk
#joblib.dump(logit, 'model/logit_group.sav')
 
# load the model from disk
#logit_group = joblib.load('model/logit_group.sav')

['model/logit_group.sav']

In [113]:
predict_group = logit_group.predict(test_embed)

Темы

In [92]:
%%time
logit_title = LogisticRegression(solver = "liblinear", C=7.0, class_weight="balanced", max_iter=200, multi_class="ovr")
logit_title.fit(tfidf_embed, y_train["Тема"])

CPU times: user 12.3 s, sys: 2.14 ms, total: 12.3 s
Wall time: 12.3 s


LogisticRegression(C=7.0, class_weight='balanced', max_iter=200,
                   multi_class='ovr', solver='liblinear')

In [93]:
# save the model to disk
#joblib.dump(logit_title, 'model/logit_title.sav')
 
# load the model from disk
#logit_title = joblib.load('model/logit_title.sav')

['model/logit_title.sav']

In [114]:
predict_title = logit_title.predict(test_embed)

Индексы

In [136]:
#группы тем
t_group = index_annoy(tfidf_embed)

In [173]:
predict_group=predict_nns(t_group, test_embed, y_train["Группа тем"], count=3)

  0%|          | 0/9743 [00:00<?, ?it/s]

In [138]:
#темы
t_title = index_annoy(tfidf_embed)

In [174]:
predict_title=predict_nns(t_group, test_embed, y_train['Тема'], count=3)

  0%|          | 0/9743 [00:00<?, ?it/s]

# Создание сабмита

In [175]:
#submission=pd.DataFrme()
#submission=test[["Текст инцидента"]].copy()
submission["Группа тем"]=predict_group
submission["Тема"]=predict_title

In [176]:
submission

,id,Группа тем,Тема
0,0,Физическая культура и спорт,Строительство спортивной инфраструктуры
1,1,Дороги,"Содержание, ремонт и обустройство тротуаров"
2,2,Благоустройство,Благоустройство придомовых территорий
3,3,Социальное обслуживание и защита,Дети и многодетные семьи
4,4,Социальное обслуживание и защита,Оказание гос. соц. помощи
...,...,...,...
9738,9738,Благоустройство,Отсутствие фонарей освещения
9739,9739,Социальное обслуживание и защита,Оказание гос. соц. помощи
9740,9740,ЖКХ,Отсутствие холодной воды
9741,9741,Социальное обслуживание и защита,Оказание гос. соц. помощи


In [177]:
submission[["id", "Группа тем", "Тема"]].to_csv("data/submission6.csv", sep=';', index=False)

In [ ]:
#0.634308 простой предикт
#0.634941 по фолдам
#0.478777 поиском
#0.480683 выбор из 3
#0.628104 логрег с поиском

In [145]:
#индексы
temp1 = submission

In [140]:
temp = submission

Постобработка

In [163]:
predict_group_new = postpredict(logit_group, t_group, test_embed, y_train["Группа тем"], lim=0.1)

In [164]:
predict_title_new = postpredict(logit_title, t_title, test_embed, y_train['Тема'], lim=0.1)

In [165]:
submission["Группа тем"]=predict_group_new
submission["Тема"]=predict_title_new

In [166]:
submission

,id,Группа тем,Тема
0,0,Физическая культура и спорт,Строительство спортивной инфраструктуры
1,1,Благоустройство,"Строительство школ, детских садов"
2,2,Дороги,Благоустройство придомовых территорий
3,3,Социальное обслуживание и защита,Дети и многодетные семьи
4,4,Социальное обслуживание и защита,Улучшение жилищных условий
...,...,...,...
9738,9738,Благоустройство,Отсутствие фонарей освещения
9739,9739,Социальное обслуживание и защита,Задержка выплат гражданам
9740,9740,Дороги,Отсутствие холодной воды
9741,9741,Социальное обслуживание и защита,Оказание гос. соц. помощи


In [167]:
submission[["id", "Группа тем", "Тема"]].to_csv("data/submission4.csv", sep=';', index=False)